In [37]:
#  import the basic libraries and configuration
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# set up display
sns.set_style("whitegrid")
sns.set_context("poster")

%matplotlib inline

In [38]:
# import knn algo
from sklearn.neighbors import KNeighborsClassifier
# import data set and metric modules
from sklearn import datasets, metrics

In [39]:
df = pd.read_csv('Dataset/diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


## Analyzing the data (including cleaning)

In [40]:
df.shape # rows and columns

(768, 9)

In [41]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


# Preprocessing the data

In [43]:
# import needed libraries for data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 

In [44]:
# get the features and predictors
features = df.columns.to_list() # get all the name of the list, good if we're going to take all the columns as a feature ( except the predictor)
features.remove('Outcome')   # remove the predictor
X = df[features]
Y = df['Outcome']

# convert to array
X = X.values
Y = Y.values.reshape(-1,1)
X.shape, Y.shape

((768, 8), (768, 1))

## Perform Normalization

In [45]:
scale= MinMaxScaler()
X = scale.fit_transform(X)
X = pd.DataFrame(X)
X.rename({0:"Pregnancies",
         1:"Glucose",
         2:"BloodPressure",
         3:"SkinThickness",
         4:"Insulin",
         5:"BMI",
         6:"DiabetesPedigreeFunction",
         7:"Age"},
         inplace=True,axis=1)

In [46]:
# see the data
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000
...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.039710,0.700000
764,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.111870,0.100000
765,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.071307,0.150000
766,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.115713,0.433333


In [47]:
# Perform train test split

x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state = 42 ,test_size = .3 , stratify = Y)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((537, 8), (231, 8), (537, 1), (231, 1))

## Train Model

In [48]:
knn = KNeighborsClassifier(n_neighbors=5)

model = knn.fit(x_train, y_train.ravel())
y_pred = model.predict(x_test)


C:\Users\johnm\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## Confusion matrix

In [49]:
#Get the Confusion Matrix and other metrics to test performance
print("Classification report for classifier %s:\n%s\n"
      % (knn, metrics.classification_report(y_test, y_pred)))

Classification report for classifier KNeighborsClassifier():
              precision    recall  f1-score   support

           0       0.75      0.81      0.78       150
           1       0.59      0.51      0.55        81

    accuracy                           0.71       231
   macro avg       0.67      0.66      0.66       231
weighted avg       0.70      0.71      0.70       231




In [50]:
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_pred))

Confusion matrix:
[[122  28]
 [ 40  41]]


## Performance of the model evaluation

In [51]:
from sklearn.metrics import roc_auc_score
## Performance of the model evaluation
roc = roc_auc_score(y_test,y_pred)
print(roc)

0.659753086419753


In [54]:
# the result
print("Actual labels:", y_test[:25].ravel())
print("Predict labels:", y_pred[:25])

Actual labels: [1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
Predict labels: [0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1]
